<a href="https://colab.research.google.com/github/1004516/SE-ALES-Y-SISTEMAS/blob/main/Solucion_Parcial_2/Simulacion_Punto_2_Parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================================
#     PUNTO 2 – SISTEMA MASA-RESORTE-AMORTIGUADOR
#
# ================================================

!pip install control --quiet

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import control
import math

# ================================================
# PARÁMETROS DEL SISTEMA FÍSICO
# ================================================
m = 1.0      # kg
k = 100.0    # N/m
omega_n = math.sqrt(k/m)

print("==============================================")
print("   PARÁMETROS DEL SISTEMA MECÁNICO")
print("==============================================")
print(f"m  = {m} kg")
print(f"k  = {k} N/m")
print(f"ωₙ = {omega_n:.4f} rad/s\n")

# CASOS: subamortiguado, crítico y sobreamortiguado
cases = {
    'Subamortiguado (c=5)': 5.0,
    'Crítico (c=20)':      20.0,
    'Sobreamortiguado (c=50)': 50.0
}

# ================================================
# FUNCIÓN DE TRANSFERENCIA MECÁNICA
# H(s) = 1 / (m s² + c s + k)
# ================================================
def H_tf(m, c, k):
    return control.TransferFunction([1], [m, c, k])

# ================================================
# CÁLCULO DE PARÁMETROS DINÁMICOS
# ================================================
def compute_params(m,c,k):
    wn = math.sqrt(k/m)
    zeta = c / (2*math.sqrt(k*m))

    if zeta < 1:
        wd = wn * math.sqrt(1 - zeta**2)
    else:
        wd = 0.0

    # Tiempos característicos
    if zeta < 1:
        Tp = np.pi / wd
        phi = math.atan(math.sqrt(1 - zeta**2)/zeta)
        tr = (np.pi - phi) / wd
    else:
        Tp = None
        tr = None

    Ts = 4/(zeta*wn)

    return {
        "zeta": zeta,
        "wn": wn,
        "wd": wd,
        "Tp": Tp,
        "tr": tr,
        "Ts": Ts
    }

# ================================================
# SIMULACIÓN
# ================================================
t = np.linspace(0, 5, 5000)

for name, c in cases.items():
    print("\n==============================================")
    print(f"     CASO: {name}")
    print("==============================================")

    # Parámetros
    p = compute_params(m, c, k)
    print(f"Amortiguamiento ζ      = {p['zeta']:.4f}")
    print(f"Frecuencia natural ωₙ  = {p['wn']:.4f} rad/s")
    print(f"Frecuencia amort. ω_d  = {p['wd']:.4f} rad/s")

    if p['Tp'] is not None:
        print(f"Tiempo al pico Tp      = {p['Tp']:.4f} s")
        print(f"Tiempo de subida t_r   = {p['tr']:.4f} s")
    print(f"Tiempo de estab. Ts    = {p['Ts']:.4f} s")

    # TF
    sys = H_tf(m, c, k)

    # Polos y ceros
    poles = sys.poles
    zeros = sys.zeros

    print("\nPolos del sistema:", poles)
    print("Ceros del sistema:", zeros)

    # BODE
    plt.figure(figsize=(12,4))
    control.bode(sys, dB=True, plot=True)
    plt.suptitle(f"Diagrama de Bode – {name}")
    plt.show()

    # RESPUESTA AL IMPULSO
    t_imp, y_imp = control.impulse_response(sys, T=t)

    plt.figure(figsize=(10,4))
    plt.plot(t_imp, y_imp)
    plt.title(f"Respuesta al Impulso – {name}")
    plt.xlabel("Tiempo [s]")
    plt.grid()
    plt.show()

    # RESPUESTA AL ESCALÓN
    t_step, y_step = control.step_response(sys, T=t)

    plt.figure(figsize=(10,4))
    plt.plot(t_step, y_step)
    plt.title(f"Respuesta al Escalón – {name}")
    plt.xlabel("Tiempo [s]")
    plt.grid()
    plt.show()

    # RESPUESTA A RAMPA (entrada = t)
    ramp = t
    t_ramp, y_ramp = control.forced_response(sys, T=t, U=ramp)

    plt.figure(figsize=(10,4))
    plt.plot(t_ramp, y_ramp)
    plt.title(f"Respuesta a Rampa – {name}")
    plt.xlabel("Tiempo [s]")
    plt.grid()
    plt.show()

    # =============================================
    # LAZO CERRADO – RETROALIMENTACIÓN UNITARIA
    # =============================================
    T_cl = control.feedback(sys, 1)

    poles_cl = T_cl.poles
    print("\nPolos en Lazo Cerrado:", poles_cl)

    # Respuesta al escalón en lazo cerrado
    t_cl, y_cl = control.step_response(T_cl, T=t)

    plt.figure(figsize=(10,4))
    plt.plot(t_cl, y_cl)
    plt.title(f"Respuesta al Escalón – Lazo Cerrado – {name}")
    plt.xlabel("Tiempo [s]")
    plt.grid()
    plt.show()

print("\n==============================================")

print("==============================================")